In [14]:
#============================================================================
#                           Importing Required Libraries
#============================================================================

import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import random
from sklearn.metrics import confusion_matrix

In [15]:
# change the directory
#os.chdir("E:\\Training\\Mindmap_Genpact\\Training_Material\\Inferential_Statistics\\Python_Hands_On")

In [16]:
#============================================================================
#              Importing Data (with text as strings and not factors)
#============================================================================

adult_try_logistic = pd.read_csv("adult.csv")
type(adult_try_logistic)
adult_try_logistic.dtypes

age                 int64
workclass          object
fnlwgt              int64
education          object
educational-num     int64
marital-status     object
occupation         object
relationship       object
race               object
gender             object
capital-gain        int64
capital-loss        int64
hours-per-week      int64
native-country     object
income             object
dtype: object

In [17]:
#============================================================================
#              Replacing "?" with NA and Encoding Dependent Variable
#                  1 = Preferred Event | 0 = Alternate Event
#============================================================================

stack = adult_try_logistic.stack()
stack[stack == "?"] = None
adult_try_logistic = stack.unstack()

adult_try_logistic.income[adult_try_logistic.income == ">50K"] = 1
adult_try_logistic.income[adult_try_logistic.income == "<=50K"] = 0

In [18]:
#============================================================================
#                       Converting text to lower case
#============================================================================

adult_try_logistic.workclass = adult_try_logistic.workclass.str.lower()
adult_try_logistic.occupation = adult_try_logistic.occupation.str.lower()
adult_try_logistic.education = adult_try_logistic.education.str.lower()
adult_try_logistic.race = adult_try_logistic.race.str.lower()
adult_try_logistic.gender = adult_try_logistic.gender.str.lower()
adult_try_logistic['native-country'] = adult_try_logistic['native-country'].str.lower()
adult_try_logistic['marital-status'] = adult_try_logistic['marital-status'].str.lower()
adult_try_logistic.relationship = adult_try_logistic.relationship.str.lower()

In [19]:
#=============================================================================
#                      Checking for % of Missing Values
# Decide whether to impute or not. If yes, with or without category wise median /           mean / mode
#                        If not, encode Missing Values
#=============================================================================

pd.isna(adult_try_logistic).mean() # gives col wise %age
type(adult_try_logistic)
adult_try_logistic.dtypes

age                object
workclass          object
fnlwgt             object
education          object
educational-num    object
marital-status     object
occupation         object
relationship       object
race               object
gender             object
capital-gain       object
capital-loss       object
hours-per-week     object
native-country     object
income             object
dtype: object

In [20]:
#=============================================================================
#                          Check for Imbalanced Data
#=============================================================================
adult_try_logistic.income.value_counts()

0    37155
1    11687
Name: income, dtype: int64

In [8]:
#=============================================================================
#                          Missing Value Replacement
#=============================================================================

adult_try_logistic.workclass[pd.isna(adult_try_logistic.workclass)]= "missing_workclass"
adult_try_logistic.occupation[pd.isna(adult_try_logistic.occupation)]= "missing_occupation"
adult_try_logistic['native-country'][pd.isna(adult_try_logistic['native-country'])]= "missing_country"

In [21]:
#=============================================================================
#                         Check Information Value (IV)
#=============================================================================
def iv_woe(data, target, bins=10, show_woe=False):

    #Empty Dataframe
    newDF = pd.DataFrame()

    #Extract Column Names
    cols = data.columns

    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
        d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
        d.columns = ['Cutoff', 'N', 'Events']
        d['% of Events'] = d['Events'] / d['Events'].sum()
        d['Non-Events'] = d['N'] - d['Events']
        d['% of Non-Events'] = d['Non-Events'] / d['Non-Events'].sum()
        d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
        d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
        print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars],
                            "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)

        #Show WOE Table
        if show_woe == True:
            print(d)
    return newDF

IV_Table = iv_woe(data = adult_try_logistic, target = 'income', bins=10, show_woe = True)

# Taken from https://www.listendata.com/2015/03/weight-of-evidence-woe-and-information.html

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\users\user\appdata\local\programs\python\python36\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Information value of age is inf
    Cutoff     N  Events  % of Events  Non-Events  % of Non-Events       WoE  \
0       17   595       0     0.000000         595         0.016014      -inf   
1       18   862       0     0.000000         862         0.023200      -inf   
2       19  1053       3     0.000257        1050         0.028260 -4.701312   
3       20  1113       1     0.000086        1112         0.029929 -5.857294   
4       21  1096       6     0.000513        1090         0.029337 -4.045552   
..     ...   ...     ...          ...         ...              ...       ...   
69      86     1       0     0.000000           1         0.000027      -inf   
70      87     3       0     0.000000           3         0.000081      -inf   
71      88     6       1     0.000086           5         0.000135 -0.452817   
72      89     2       0     0.000000           2         0.000054      -inf   
73      90    55      13     0.001112          42         0.001130 -0.016099   

       

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Information value of fnlwgt is inf
        Cutoff  N  Events  % of Events  Non-Events  % of Non-Events       WoE  \
0        12285  1       0     0.000000           1         0.000027      -inf   
1        13492  1       0     0.000000           1         0.000027      -inf   
2        13769  3       1     0.000086           2         0.000054  0.463474   
3        13862  1       0     0.000000           1         0.000027      -inf   
4        14878  1       1     0.000086           0         0.000000       inf   
...        ... ..     ...          ...         ...              ...       ...   
28518  1268339  1       0     0.000000           1         0.000027      -inf   
28519  1366120  1       0     0.000000           1         0.000027      -inf   
28520  1455435  1       0     0.000000           1         0.000027      -inf   
28521  1484705  1       0     0.000000           1         0.000027      -inf   
28522  1490400  1       0     0.000000           1         0.000027      -

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\users\user\appdata\local\programs\python\python36\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Information value of capital-gain is inf
     Cutoff      N  Events  % of Events  Non-Events  % of Non-Events  \
0         0  44807    9196     0.786857       35611         0.958444   
1       114      8       0     0.000000           8         0.000215   
2       401      5       0     0.000000           5         0.000135   
3       594     52       0     0.000000          52         0.001400   
4       914     10       0     0.000000          10         0.000269   
..      ...    ...     ...          ...         ...              ...   
118   25236     14      14     0.001198           0         0.000000   
119   27828     58      58     0.004963           0         0.000000   
120   34095      6       0     0.000000           6         0.000161   
121   41310      3       0     0.000000           3         0.000081   
122   99999    244     244     0.020878           0         0.000000   

          WoE        IV  
0   -0.197265  0.033848  
1        -inf       inf  
2        -inf   

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\users\user\appdata\local\programs\python\python36\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Information value of native-country is inf
                        Cutoff      N  Events  % of Events  Non-Events  \
0                     cambodia     28       9     0.000785          19   
1                       canada    182      63     0.005494         119   
2                        china    122      36     0.003139          86   
3                     columbia     85       4     0.000349          81   
4                         cuba    138      34     0.002965         104   
5           dominican-republic    103       5     0.000436          98   
6                      ecuador     45       6     0.000523          39   
7                  el-salvador    155      11     0.000959         144   
8                      england    127      47     0.004099          80   
9                       france     38      16     0.001395          22   
10                     germany    206      58     0.005058         148   
11                      greece     49      18     0.001570          3

In [10]:
#============================================================================
#                             Thumb Rule for IV
#============================================================================
#Information Value	Predictive Power
#< 0.02	useless for prediction
#0.02 to 0.1	Weak predictor
#0.1 to 0.3	Medium predictor
#0.3 to 0.5	Strong predictor
#>0.5	Suspicious or too good to be true

In [11]:
#=============================================================================

#=============================================================================
#                          Converting back to Categorical Variables
#=============================================================================
adult_try_logistic.workclass = adult_try_logistic.workclass.astype('category')
adult_try_logistic.occupation = adult_try_logistic.occupation.astype('category')
adult_try_logistic['native-country'] = adult_try_logistic['native-country'].astype('category')
adult_try_logistic.race = adult_try_logistic.race.astype('category')
adult_try_logistic.gender = adult_try_logistic.gender.astype('category')
adult_try_logistic.relationship = adult_try_logistic.relationship.astype('category')
adult_try_logistic.income = adult_try_logistic.income.astype('category')
adult_try_logistic.education = adult_try_logistic.education.astype('category')
adult_try_logistic['marital-status'] = adult_try_logistic['marital-status'].astype('category')
adult_try_logistic.dtypes

age                  object
workclass          category
fnlwgt               object
education          category
educational-num      object
marital-status     category
occupation         category
relationship       category
race               category
gender             category
capital-gain         object
capital-loss         object
hours-per-week       object
native-country     category
income             category
dtype: object

In [ ]:
#                      Choosing Relevant Independent Variables
#=============================================================================

adult_try_logistic = adult_try_logistic.drop(axis=1,columns=['fnlwgt','race','capital-loss','native-country'])

In [ ]:
#============================================================================
#                           Create Training Data (Bootstrapping)
#============================================================================
adult_try_logistic = adult_try_logistic.rename(columns={"hours-per-week":"hours_per_week"})
adult_try_logistic.age = adult_try_logistic.age.astype('int')
adult_try_logistic['educational-num'] = adult_try_logistic['educational-num'].astype('int')
adult_try_logistic['capital-gain'] = adult_try_logistic['capital-gain'].astype('int')
adult_try_logistic.hours_per_week = adult_try_logistic.hours_per_week.astype('int')


input_ones = adult_try_logistic[adult_try_logistic.income == 1]  # all 1's code (encoding) of whichever level is lower in frequency
input_zeroes = adult_try_logistic[adult_try_logistic.income == 0]  # all 0's
random.seed(5000)
training_ones = input_ones.sample(frac=0.7)
training_zeroes = input_zeroes.sample(frac=0.7)
trainingData = pd.concat([training_ones,training_zeroes])

In [ ]:
#===========================================================================
#                               Create Test Data
#===========================================================================
test_ones = input_ones.loc[~input_ones.index.isin(training_ones.index)]
test_zeroes = input_zeroes.loc[~input_zeroes.index.isin(training_zeroes.index)]
testData = pd.concat([test_ones,test_zeroes])  # row bind the 1's and 0's

In [ ]:
#===========================================================================
#                      Modeling (Tuning based on p-values)
#                  Number of Fisher Iterations should be less
#===========================================================================
logitMod = smf.glm('income ~ age + C(workclass) + C(education) + C(occupation) + hours_per_week', data=trainingData, family=sm.families.Binomial()).fit()
logitMod.summary()

In [ ]:
#===========================================================================
#                            Testing (Prediction)
#===========================================================================

predicted = logitMod.predict(testData[['age','workclass','education','occupation','hours_per_week']])

In [ ]:
#===========================================================================
#                          Deciding Optimal Cut-off
# Cut-off is the point above which it is 1 (Preferred Event) and below it, is 0 (Alternate Event)
# Ideal cut off is 0.5 which means if the prediction is 0.4 or 40% it is an Alternate Event, but sometimes due to imbalance
# in your dataset, the cut-off is higher say 0.92 which means if your prediction is 0.88 it is still Alternate Event.
#===========================================================================

from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds =roc_curve(testData.income, predicted)
roc_auc = auc(fpr, tpr)
print("Area under the ROC curve : %f" % roc_auc)

i = np.arange(len(tpr)) # index for df
roc = pd.DataFrame({'fpr' : pd.Series(fpr, index=i),'tpr' : pd.Series(tpr, index = i), '1-fpr' : pd.Series(1-fpr, index = i), 'tf' : pd.Series(tpr - (1-fpr), index = i), 'thresholds' : pd.Series(thresholds, index = i)})

In [ ]:
# Threshold
optCutOff = roc.ix[(roc.tf-0).abs().argsort()[:1]]

In [ ]:
# Plot tpr vs 1-fpr
fig, ax = plt.subplots()
plt.plot(roc['tpr'])
plt.plot(roc['1-fpr'], color = 'red')
plt.xlabel('1-False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
ax.set_xticklabels([])
plt.show()

In [ ]:
#============================================================================
#                        Checking Accuracy of the Model
#============================================================================

fitted_results = predicted
for k in fitted_results.index:
    print(k)
    if fitted_results[k] >= optCutOff['thresholds'].values:
        fitted_results[k]=1
    else:
        fitted_results[k]=0

misClasificError = np.mean(fitted_results != testData.income)
print('Accuracy',1-misClasificError)

In [ ]:
#============================================================================
#                               Confusion Matrix
#============================================================================

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

results = confusion_matrix(testData.income, fitted_results)
print('Confusion Matrix :')
print(results)
print('Accuracy Score :',accuracy_score(testData.income, fitted_results))
print('Report : ')
print(classification_report(testData.income, fitted_results))

tn, fp, fn, tp = results.ravel()

In [ ]:
#============================================================================
#                                Sensitivity
#============================================================================
print(tp/(tp+fn))

In [ ]:
#============================================================================
#                                Specificity
#============================================================================
print(tn/(tn+fp))